In [3]:
import os
import cv2 as cv
import common as c
import pandas as pd

In [49]:
def dhash(img, size=8):
    row_hash = 0
    col_hash = 0
    for y in range(size):
        for x in range(size):
            row_bit = img[y,x] < img[y,x + 1] 
            row_hash = row_hash << 1 | row_bit
            
            col_bit = img[y,x] < img[y + 1,x]
            col_hash = col_hash << 1 | col_bit
    return format_hex(row_hash, col_hash, size)

# Reference Dhash
# https://github.com/benhoyt/dhash
# https://benhoyt.com/writings/duplicate-image-detection/
#
def format_hex(row_hash, col_hash, size=8):
    """Format dhash integers as hex string of size*size//2 total hex digits
    (row_hash and col_hash concatenated).
    >>> format_hex(19409, 14959, size=4)
    '4bd13a6f'
    >>> format_hex(1, 2, size=4)
    '00010002'
    """
    hex_length = size * size // 4
    return '{0:0{2}x}{1:0{2}x}'.format(row_hash, col_hash, hex_length)


In [54]:
def hash_img(batch_no, batch_list):
    hash_list = []
    for i,img_path in batch_list:
        mat_img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
        img = cv.resize(mat_img, (9, 9), interpolation = cv.INTER_AREA)
        h_1 = dhash(img)
        hash_lsit.append(h_1)
    df = pd.DataFrame({"img":batch_list, "hash": hash_list})
    df.to_csv(os.path.join(c.HASH_PATH), 'b_' + str(batch_no) + "_hash.csv")

-33323b3606766dad00000000c0098876 5cd4d591191832b2-5071b28fec6cc82


In [ ]:
print("Total :", len(img_list))

process_list = []
p_count = multiprocessing.cpu_count()

element = math.floor(len(img_list)/p_count)

for p_c in range(p_count):
    start = p_c * element
    end = (p_c + 1) * element
    if p_c == p_count - 1:
        end += len(img_list) % p_count
    print("D - ", p_c, ":",start, end)
    p = multiprocessing.Process(target=hash_img, args=(p_c, img_list[start:end], ))
    p.start()
    process_list.append(p)

for p in process_list:
    p.join()

print("Completed.");

# Join into Master Hash

In [9]:
dbs = os.listdir(c.HASH_PATH)
dfs = [pd.read_csv(os.path.join(c.HASH_PATH,x), index_col=0) for x in dbs if x[0] != 'M']
master_df = pd.concat(dfs, ignore_index=True)

In [10]:
master_df.to_csv(c.HASH_DB)
dbs = [os.remove(os.path.join(c.HASH_PATH,x)) for x in dbs]

# Check Duplicates

In [11]:
master_df['hash'].value_counts

<bound method IndexOpsMixin.value_counts of 0        -6543c2adc8e4bce31eb079c304a04611
1        -53182c286aad2c8b116802c3bc101ec1
2         3eaa2a266aeb26367596c825780bc3c0
3        3556d3c676b8c854-168f1fe1b9de77a4
4        -104c6d6d4d6b6b6b080001022668841d
                       ...                
4895      6d55a88839bb2e682e5460880013bf28
4896      5ae869e9ede4e4ca1e16737ba9000580
4897      572a3a6aba3665ac556d344a8812450c
4898     -25a5a12898dad2dd100358104604b9ff
4899    -62c3c1e1c3436264-70a603e8099ffb01
Name: hash, Length: 4900, dtype: object>

In [13]:
dff = pd.DataFrame({'c': [1, 2,3,4,1]})
dff['c'].value_counts

<bound method IndexOpsMixin.value_counts of 0    1
1    2
2    3
3    4
4    1
Name: c, dtype: int64>